Here we create RMS signals following next steps:
1. Preprocessing with LP filter with cut-off frequency of 3000 Hz
2. Downsampling signals to 10 kHz
3. Band-pass squared filtered signal with fl=2 Hz and fh=100 Hz
4. Binnning the signal into 25 ms long windows and calculating squared root of mean value

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Three possible scenarios:
* session: 1,2,3,4,5 or False - where false stands for inter-session approach
* reduced: True = take first 200 ms of each repetition and each class; False = take entire 1 s of each
* if session = False, we can cerate randomly, or non-randomly choosen sets

In [ ]:
# FILL IN PATHS

session=1
reduced=True #true=200 ms, false=1s
inter_session_randomly=True

path=['','','']
if reduced:
  path[0]='/content/gdrive/Shared drives/Nikolina/RMS data/1D features/Inter-session/RMS_train_200ms.csv'
  path[1]='/content/gdrive/Shared drives/Nikolina/RMS data/1D features/Inter-session/RMS_valid_200ms.csv'
  path[2]='/content/gdrive/Shared drives/Nikolina/RMS data/1D features/Inter-session/RMS_test_200ms.csv'
else:
  path[0]='/content/gdrive/Shared drives/Nikolina/RMS data/1D features/Inter-session/RMS_train_1s.csv'
  path[1]='/content/gdrive/Shared drives/Nikolina/RMS data/1D features/Inter-session/RMS_valid_1s.csv'
  path[2]='/content/gdrive/Shared drives/Nikolina/RMS data/1D features/Inter-session/RMS_test_1s.csv'

In [ ]:
import h5py
import pandas as pd
import numpy as np
from skimage import io
import scipy
import scipy.signal as signal
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
def read_data(session_numb):
    """Function loads mat files, depending on session number from 1 to 5. 
    Note that it paths to mat files should be updated!
    Returns:
    sig_out - signals with shape: number of samples x 56
    targets_out - regression targets with shape: number of samples x 5 (for 5 fingers), 
    targets_out10 - the same as targets_out, but with fs=10 kHz, instead of 30 kHz. This
    one are going to be used as images will be downsampled as well.

    ------

    NOTE: There is a difference between read_data in this, and other scripts, because for NN 
    approach we use cleaned signals from session 3 (as artifact peak was causing problems). So we just read 
    sig_cleaned and corresponding targets. 
    """

    day16name='/content/gdrive/Shared drives/Nikolina/19.5.2020./Data/Data10Khz/session1_10kHz.mat'
    day17name='/content/gdrive/Shared drives/Nikolina/19.5.2020./Data/Data10Khz/session2_10kHz.mat'
    day20name='/content/gdrive/Shared drives/Nikolina/19.5.2020./Data/Data10Khz/session3_10kHz.mat'
    day23name1='/content/gdrive/Shared drives/Nikolina/19.5.2020./Data/Data10Khz/session4_10kHz.mat'
    day23name2='/content/gdrive/Shared drives/Nikolina/19.5.2020./Data/Data10Khz/session5_10kHz.mat'

    
    c1=len(session_numb)
    c2=session_numb[0]
       
    if (1 in session_numb):
        fulldata     = h5py.File(day16name)
        #trigger=fulldata["trigger"][:]
        sig=fulldata["sig"][:]
        #t=fulldata['time'][:]
        targets=fulldata['targets'][:]
        
        targets10=fulldata['targets10'][:]
        targets10c=fulldata['targets10c'][:]
        if c1==1 or c2==1:
            sig_out=sig
            targets_out=targets
            targets_out10=targets10
            targets_out10c=targets10c
        else:
            sig_out=np.concatenate((sig_out,sig),axis=0)
            targets_out=np.concatenate((targets_out,targets),axis=0)
            targets_out10=np.concatenate((targets_out10,targets10),axis=0)
            targets_out10c=np.concatenate((targets_out10c,targets10c),axis=0)


    if (2 in session_numb):
        fulldata     = h5py.File(day17name)
        #trigger=fulldata["trigger"][:]
        sig=fulldata["sig"][:]
        #t=fulldata['time'][:]
        targets=fulldata['targets'][:]

        targets10=fulldata['targets10'][:]
        targets10c=fulldata['targets10c'][:]
        if c1==1 or c2==2:
            sig_out=sig
            targets_out=targets
            targets_out10=targets10
            targets_out10c=targets10c
        else:
            sig_out=np.concatenate((sig_out,sig),axis=0)
            targets_out=np.concatenate((targets_out,targets),axis=0)
            targets_out10=np.concatenate((targets_out10,targets10),axis=0)
            targets_out10c=np.concatenate((targets_out10c,targets10c),axis=0)

        
    if (3 in session_numb):
        fulldata     = h5py.File(day20name)
        #trigger=fulldata["trigger"][:]
        sig=fulldata["sig_cleaned"][:]#sifnals from session3 are additionaly cleaned!
        #t=fulldata['time'][:]
        targets=fulldata['targets_cleaned'][:]

        targets10=fulldata['targets10_cleaned'][:]
        targets10c=fulldata['targets10c_cleaned'][:]
        if c1==1 or c2==3:
            sig_out=sig
            targets_out=targets
            targets_out10=targets10
            targets_out10c=targets10c
        else:
            sig_out=np.concatenate((sig_out,sig),axis=0)
            targets_out=np.concatenate((targets_out,targets),axis=0)
            targets_out10=np.concatenate((targets_out10,targets10),axis=0)
            targets_out10c=np.concatenate((targets_out10c,targets10c),axis=0)
            
    if (4 in session_numb):
        fulldata     = h5py.File(day23name1)
        #trigger=fulldata["trigger"][:]
        sig=fulldata["sig"][:]
        #t=fulldata['time'][:]
        targets=fulldata['targets'][:]

        targets10=fulldata['targets10'][:]
        targets10c=fulldata['targets10c'][:]
        if c1==1 or c2==4:
            sig_out=sig
            targets_out=targets
            targets_out10=targets10
            targets_out10c=targets10c
        else:
            sig_out=np.concatenate((sig_out,sig),axis=0)
            targets_out=np.concatenate((targets_out,targets),axis=0)
            targets_out10=np.concatenate((targets_out10,targets10),axis=0)
            targets_out10c=np.concatenate((targets_out10c,targets10c),axis=0)
       
        
    if (5 in session_numb):
        fulldata     = h5py.File(day23name2)
        #trigger=fulldata["trigger"][:]
        sig=fulldata["sig"][:]
        #t=fulldata['time'][:]
        targets=fulldata['targets'][:]

        targets10=fulldata['targets10'][:]
        targets10c=fulldata['targets10c'][:]
        if c1==1 or c2==5:
            sig_out=sig
            targets_out=targets
            targets_out10=targets10
            targets_out10c=targets10c
        else:
            sig_out=np.concatenate((sig_out,sig),axis=0)
            targets_out=np.concatenate((targets_out,targets),axis=0)
            targets_out10=np.concatenate((targets_out10,targets10),axis=0)
            targets_out10c=np.concatenate((targets_out10c,targets10c),axis=0)
     
    return sig_out, targets_out, targets_out10, targets_out10c


def create_rms(data,targets,w,c): 
    """Function creates windows of desired window length and 0 overlap from signals.
    Then it computes square of mean for each window and creates corresponding target by taking 
    the last target fom the window.
    Returns:
    inputs - now each window is represented with one RMS list.
    outputs - now each window is represented with one target list.
    """ 
    fs=10000
    w=int(w*fs) #window length 
    o=0

    if c:
      multiply=1
    else: #if regression multiply targets by factor 10
      multiply=10

    inputs=[]
    outputs=[]
    input0=rmValue(data[0:56,0:w])
    inputs.append(input0)
    output0=multiply*targets[:,w]
    outputs.append(output0)
    i=w-o
    last=np.shape(data)[1]-w-1
    while i<last:

        inputi=data[0:56,i:i+w]
        rms=rmValue(inputi)
        #inputi=Normalizer().fit_transform(inputi)
        inputs.append(rms)
        targeti=multiply*targets[:,i+w]
        outputs.append(targeti)
        i=i+w-o

    inputs.append(rmValue(data[0:56,last:last+w]))
    outputs.append(multiply*targets[:,last+w])
    inputs=np.array(inputs)
    outputs=np.array(outputs)
    print('input: ',np.shape(inputs))
    print('output: ',np.shape(outputs))
    return inputs, outputs

def rmValue(data): 
  """Loop over 56 channels, 
  calculate mean and square it.
  Returns 1 x 56 vector. """
  n=np.shape(data)[0]
  RMS=np.zeros(n)
  for i in range(0,n): 
    RMS[i] = np.sqrt(np.mean(data[i,:]**2)) 

  return RMS 

 # Read session (always one).

In [ ]:
(t_data, t_targets, downs_targets,downs_targetsc)=read_data([session])# training sessions
print('data: ', np.shape(t_data))
print('targets: ', np.shape(t_targets))
print('targets10: ', np.shape(downs_targets))
print('targets10c: ', np.shape(downs_targetsc))
t_data=np.transpose(t_data)
t_targets=np.transpose(t_targets)
downs_targets=np.transpose(downs_targets)
downs_targetsc=np.transpose(downs_targetsc)
print('transposed data: ', np.shape(t_data))
print('transposed targets: ', np.shape(t_targets))
print('transposed targets10: ', np.shape(downs_targets))
print('transposed targets10c: ', np.shape(downs_targetsc))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.


data:  (5550000, 56)
targets:  (5550000, 5)
targets10:  (1850000, 5)
targets10c:  (1850000, 9)
transposed data:  (56, 5550000)
transposed targets:  (5, 5550000)
transposed targets10:  (5, 1850000)
transposed targets10c:  (9, 1850000)


# Step 1 - filtering.

In [ ]:
fs=30000
high=3000/fs
low=300/fs
#b2, a2 = signal.butter(4, [low,high], btype='bandpass')
b2, a2 = signal.butter(4, high, btype='lowpass')
n=np.shape(t_data)[1]
filtered=np.zeros((56,n))

print(' [INFO] Filtering ..')
for channel in tqdm(np.arange(0,56,1)): 
  filtered[channel,:] = signal.filtfilt(b2, a2,t_data[channel,:]).reshape(1,n)
t_data=filtered

#channel=51
#plt.figure(figsize=(20,8))
#plt.plot(t_data[channel,:])
#plt.title('Filtered signal, channel'+str(channel))

  0%|          | 0/56 [00:00<?, ?it/s]

 [INFO] Filtering ..


100%|██████████| 56/56 [00:09<00:00,  5.85it/s]


# Step 2 - downsampling.

In [ ]:
downs_data=np.zeros((56,int(np.shape(t_data)[1]/3)))
for channel in range(0,56):
  downs_data[channel,:]=scipy.signal.decimate(t_data[channel,:], 3)

print('original=',np.shape(t_data))
print('downsampled=',np.shape(downs_data))
print('targets: ', np.shape(downs_targets))
print('targetsc: ', np.shape(downs_targetsc))


"""
channel=51
plt.figure(figsize=(20,8))
plt.plot(downs_data[channel,:])
plt.title('Filtered signal, channel'+str(channel))
"""

original= (56, 5550000)
downsampled= (56, 1850000)
targets:  (5, 1850000)
targetsc:  (9, 1850000)


"\nchannel=51\nplt.figure(figsize=(20,8))\nplt.plot(downs_data[channel,:])\nplt.title('Filtered signal, channel'+str(channel))\n"

# Steps 3 and 4 --> result is binned RMS with fs = 40 Hz

In [ ]:
fs=10000
high=100/fs
low=2/fs
#b2, a2 = signal.butter(2, high, btype='lowpass',analog=False)
b2, a2 = signal.butter(2, [low,high] , btype='bandpass')
n=np.shape(downs_data)[1]
envelope=np.zeros((56,n))

#filt = signal.butter(2, 100, 'lowpass', fs=10000, output='sos', analog=False)
#filtered=signal.sosfilt(filt, downs_data[0,:]**2).reshape(1,n)

print('Filtering ...')
for channel in tqdm(np.arange(0,56,1)): 
  envelope[channel,:] = signal.filtfilt(b2, a2,downs_data[channel,:]**2).reshape(1,n)


# 1000/250 = 40 Hz new sampling frequency
w=0.025
[inputs, outputs]=create_rms(envelope,downs_targetsc,w,1)#1-classification

#plt.figure(figsize=(20,8))
#plt.plot(inputs[:,0])

#plt.figure(figsize=(20,8))
#plt.plot(outputs[:,0])

#plt.figure(figsize=(20,8))
#plt.plot(inputs[0:2*40,19,0],linewidth=3)

  0%|          | 0/56 [00:00<?, ?it/s]

Filtering ...


100%|██████████| 56/56 [00:02<00:00, 23.70it/s]


input:  (7400, 56)
output:  (7400, 9)


________________________________________________________________________________

Now create vectors that represent starting time point for each repetition, of each class and each session. (This is done manually) <br>
For rest - we take subset of the repetitions such that classes are balanced.

In [ ]:
#extract first 200 ms or 1 ms - starting point
c=[]
if session ==1:
  rest=np.array([2,7,12,52,57,102,107,152,157,162])
  c.append(rest)
  c.append(np.arange(0,50,5))
  c.append(np.arange(1,51,5))
  c.append(np.arange(50,100,5))
  c.append(np.arange(51,101,5))
  c.append(np.arange(100,150,5))
  c.append(np.arange(101,151,5))
  c.append(np.arange(150,185,5))
  c.append(np.arange(151,186,5))

elif session==2:
  rest=np.array([2,7,12,52,57,102,107,152,157,162])
  c.append(rest)
  c.append(np.arange(0,50,5))
  c.append(np.arange(1,51,5))
  c.append(np.arange(50,90,5))
  c.append(np.arange(51,91,5))
  c.append(np.arange(90,140,5))
  c.append(np.arange(91,141,5))
  c.append(np.arange(140,190,5))
  c.append(np.arange(141,191,5))

elif session==3:
  rest=np.array([2,7,12,52,57,102,107,152,157,162])
  c.append(rest)
  c.append(np.arange(0,50,5))
  c.append(np.arange(1,51,5))
  c.append(np.arange(50,100,5))
  c.append(np.arange(51,101,5))
  c.append(np.arange(100,150,5))
  c.append(np.arange(101,151,5))
  c.append(np.arange(150,195,5))
  c.append(np.arange(151,196,5))


else:
  rest=np.array([2,7,12,52,57,102,107,152,157,162])
  c.append(rest)
  c.append(np.arange(0,50,5))
  c.append(np.arange(1,51,5))
  c.append(np.arange(50,100,5))
  c.append(np.arange(51,101,5))
  c.append(np.arange(100,150,5))
  c.append(np.arange(101,151,5))
  c.append(np.arange(150,200,5))
  c.append(np.arange(151,201,5))

print(c)


[array([  2,   7,  12,  52,  57, 102, 107, 152, 157, 162]), array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45]), array([ 1,  6, 11, 16, 21, 26, 31, 36, 41, 46]), array([50, 55, 60, 65, 70, 75, 80, 85, 90, 95]), array([51, 56, 61, 66, 71, 76, 81, 86, 91, 96]), array([100, 105, 110, 115, 120, 125, 130, 135, 140, 145]), array([101, 106, 111, 116, 121, 126, 131, 136, 141, 146]), array([150, 155, 160, 165, 170, 175, 180]), array([151, 156, 161, 166, 171, 176, 181])]


Now extract just 200 ms of the single repetition (reduced = True), or 1 s (reduced=False).

In [ ]:
if reduced:
  nb_of_s=8
else:
  nb_of_s=40

s=0
i=0
for l in c:
  f=0
  for begin in l:
    if i==0 and f==0:
      features=inputs[int(begin*40):int(begin*40)+nb_of_s,:].T
      labels=outputs[int(begin*40):int(begin*40)+nb_of_s,s:9].T
    else:
      features=np.concatenate((features,inputs[int(begin*40):int(begin*40)+nb_of_s,:].T),axis=1)
      labels=np.concatenate((labels,outputs[int(begin*40):int(begin*40)+nb_of_s,s:9].T),axis=1)
    f+=1
    #print(outputs[int(begin*40)+2:int(begin*40)+10,s:9].T)
  i+=1

features=features.T
labels=labels.T

print(np.shape(features))
print(np.shape(labels))

(672, 56)
(672, 9)


________________________________________________________________________________

# RANDOM SPLIT

In [ ]:
#first split - train+validation versus test
X, X_test, y, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
print(np.shape(X))
#second split - train versus validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)
print('train data: ', np.shape(X_train),np.shape(y_train))
print('validation data: ',np.shape(X_val),np.shape(y_val))
print('test data: ',np.shape(X_test),np.shape(y_test))

(537, 56)
train data:  (402, 56) (402, 9)
validation data:  (135, 56) (135, 9)
test data:  (135, 56) (135, 9)


# STANDARDIZATION

Extract mean and standard deviation from training set, and normalize data according to it.

In [ ]:
m1=np.std(X_train,axis=0)
m2=np.mean(X_train,axis=0)
print(np.shape(m1))
print(np.shape(m2))

X_train=np.divide(X_train-m2,m1)#normalize train
X_val=np.divide(X_val-m2,m1)#normalize validation
X_test=np.divide(X_test-m2,m1)#normalize test

print('train data: ', np.shape(X_train))
print('validation data: ',np.shape(X_val))
print('test data: ',np.shape(X_test))

(56,)
(56,)
train data:  (402, 56)
validation data:  (135, 56)
test data:  (135, 56)


# CREATING THE DATAFRAMES AND SAVING 

In [ ]:
e=9
df_signals=pd.concat([pd.DataFrame([X_train[:,i]]) for i in range(56)],
          ignore_index=True)

df_targets=pd.concat([pd.DataFrame([y_train[:,i]]) for i in range(e)],
          ignore_index=True)

df_final=df_signals.append(df_targets)
df_final.to_csv(path[0],index=False)
print(path[0],df_final) 

/content/gdrive/Shared drives/Nikolina/RMS data/1D features/Inter-session/RMS_train_rs_200ms.csv         0         1         2     ...      2099      2100      2101
0  -0.240286 -0.235804 -0.128265  ... -0.181038 -0.241764  0.075499
1  -0.256663  1.349196 -0.250609  ... -0.201371  0.727993  0.296259
2   0.299472 -0.245906 -0.357011  ... -0.357227 -0.354241 -0.353658
3  -0.151290 -0.148755 -0.136339  ... -0.131999 -0.151212 -0.070860
4  -0.217581 -0.217570 -0.096181  ... -0.190058 -0.216594  0.059085
..       ...       ...       ...  ...       ...       ...       ...
4   0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000
5   0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000
6   0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000
7   0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000
8   1.000000  0.000000  0.000000  ...  1.000000  0.000000  0.000000

[65 rows x 2102 columns]


In [ ]:
df_signals=pd.concat([pd.DataFrame([X_val[:,i]]) for i in range(56)],
          ignore_index=True)

df_targets=pd.concat([pd.DataFrame([y_val[:,i]]) for i in range(e)],
          ignore_index=True)

df_final=df_signals.append(df_targets)
df_final.to_csv(path[1],index=False)
print(path[1],df_final) 

/content/gdrive/Shared drives/Nikolina/RMS data/1D features/Inter-session/RMS_valid_rs_200ms.csv          0         1         2    ...       698       699       700
0  -0.199164 -0.146913 -0.216228  ... -0.233634 -0.240124  0.105071
1  -0.236301 -0.243342 -0.241075  ... -0.258803 -0.256076 -0.256421
2  -0.351007 -0.349146 -0.345089  ...  0.509606 -0.113410 -0.355233
3  -0.045312  0.010213 -0.119230  ... -0.150281 -0.151078 -0.093047
4  -0.182342 -0.158021 -0.196735  ... -0.212197 -0.220542 -0.033340
..       ...       ...       ...  ...       ...       ...       ...
4   0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000
5   0.000000  0.000000  0.000000  ...  1.000000  0.000000  0.000000
6   1.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000
7   0.000000  1.000000  0.000000  ...  0.000000  0.000000  0.000000
8   0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000

[65 rows x 701 columns]


In [ ]:
df_signals=pd.concat([pd.DataFrame([X_test[:,i]]) for i in range(56)],
          ignore_index=True)

df_targets=pd.concat([pd.DataFrame([y_test[:,i]]) for i in range(e)],
          ignore_index=True)

df_final=df_signals.append(df_targets)
df_final.to_csv(path[2],index=False)
print(path[2],df_final) 

/content/gdrive/Shared drives/Nikolina/RMS data/1D features/Inter-session/RMS_test_rs_200ms.csv          0         1         2    ...       698       699       700
0  -0.218651  0.043339 -0.237437  ... -0.239574 -0.232223 -0.222218
1  -0.258520 -0.250907 -0.258796  ... -0.258883 -0.258385 -0.258938
2  -0.031618 -0.351594  0.479333  ...  0.161223  0.478103  0.498135
3  -0.145544 -0.084332 -0.149297  ... -0.151519 -0.149586 -0.145334
4  -0.210088  0.007916 -0.212100  ... -0.215379 -0.205853 -0.197720
..       ...       ...       ...  ...       ...       ...       ...
4   0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000
5   0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000
6   0.000000  0.000000  0.000000  ...  0.000000  1.000000  0.000000
7   0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000
8   0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000

[65 rows x 701 columns]
